In [ ]:
import pandas as pd
from difflib import SequenceMatcher
import json
coursera_df = pd.read_excel('course_data.xlsx', sheet_name='Course Data')
coursera_df = coursera_df.dropna(subset=['skills'])
coursera_df = coursera_df.dropna(subset=['difficulty'])

tsc_ccs_key_df = pd.read_excel('SkillsFramework_Dataset_2024_06.xlsx', sheet_name='TSC_CCS_Key')
jobs_skillframework = pd.read_excel('SkillsFramework_Dataset_2024_06.xlsx', sheet_name='Job Role_Description')


In [ ]:
tsc_ccs_key_skills_df = tsc_ccs_key_df['TSC_CCS Title']
coursera_skills_df = coursera_df['skills']

In [ ]:
coursera_skills_set = set()
for row in coursera_skills_df:
    coursera_skills_list = [item.strip() for item in row.split(',')]
    for skill in coursera_skills_list:
        coursera_skills_set.add(skill)

coursera_skills_set

In [ ]:
tsc_ccs_key_skills_df = tsc_ccs_key_skills_df.drop_duplicates()

In [ ]:
def trigram_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
threshold = 0.7  # Define a similarity threshold
# Dictionary to store the best match of each Coursera skill to a TSC skill
best_match_skills_map = {}
less_than_threshold_map = {}

for skill in coursera_skills_set:
    # Split the skills string from Coursera and strip any extra spaces
    max_similarity = 0  # Reset for each skill
    best_match = ['', 0]  # To store the best match result for each skill

    for tsc_skill in tsc_ccs_key_skills_df:
        similarity_score = trigram_similarity(skill, tsc_skill)
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            best_match = [tsc_skill, max_similarity]
    
    if best_match[1] <= threshold:
        less_than_threshold_map[skill] = best_match
    
    # Print the best match found for the current Coursera skill
    print(f"coursera_skill: {skill} similarity score: {best_match[1]} to tsc_skill: {best_match[0]}")
    
    # Map each Coursera skill to its best match regardless of the threshold
    best_match_skills_map[skill] = best_match


In [ ]:
threshold = 0.85  # Define a similarity threshold
# Dictionary to store the best match of each Coursera skill to a TSC skill
skillframe_tsc_similarity_coursera = {}
skillframe_lessthansimilarity_tsc_similarity_coursera = {}
for tsc_skill in tsc_ccs_key_skills_df:
    max_similarity = 0  # Reset for each skill
    best_match = ['', 0]  # To store the best match result for each skill
    for skill in coursera_skills_set:
        
        similarity_score = trigram_similarity(skill, tsc_skill)
        
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            best_match = [skill, max_similarity]
        print(f"{tsc_skill}, {skill}, {best_match}")
    if best_match[1] <= threshold:
        skillframe_lessthansimilarity_tsc_similarity_coursera[tsc_skill] = best_match
    
    skillframe_tsc_similarity_coursera[tsc_skill] = best_match

In [ ]:
import os
folder_path = 'coursera'
try:
    os.mkdir(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{folder_path}' already exists.")
except Exception as e:
    print(f"An error occurred: {e}")


with open('./coursera/best_coursera_match_skills_map.json', 'w') as json_file:
    json.dump(best_match_skills_map, json_file, indent=4)
    print('Successfully save best_match_skills_map as json')

with open('./coursera/less_than_threshold_map.json', 'w') as json_file:
    json.dump(less_than_threshold_map , json_file, indent=4)
    print('Successfully save less_than_threshold_map as json')

with open('./coursera/skillframe_tsc_similarity_coursera.json', 'w') as json_file:
    json.dump(skillframe_tsc_similarity_coursera, json_file, indent=4)
    print('Successfully save skillframe_tsc_similarity_coursera as json')

with open('./coursera/skillframe_lessthansimilarity_tsc_similarity_coursera.json', 'w') as json_file:
    json.dump(skillframe_lessthansimilarity_tsc_similarity_coursera, json_file, indent=4)
    print('Successfully save skillframe_lessthansimilarity_tsc_similarity_coursera as json')

